# Project: Finding Correlations among Unrelated Variables

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploring the Data</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## I) Introduction

**Broad question:** How do total forest area and frequency of natural disasters shape a country's obesity rates and murder rates?

> I picked these factors which seem to be unrelated--geography & frequency of natural disasters, to rates of obesity murder, to formulate new interesting questions and uncover unexpected patterns. I also wanted to approach this project through an experimental and free-for-all lens, just to see if I can make any fun or comical conclusions from the giving unrelated datasets. To narrow down my focus, I picked subcategory of [TK] for geography, [TK] for education and [TK] to encompass murder rates. I received all of my data through GapMinder.

In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<a id='wrangling'></a>
# II) Data Wrangling

## A) Gathering Data

#### Natural Disasters
> I took [TK list dataset names here], each depicting the number of deaths of their respective natural disaster. To combine those datasets, I generated a new CSV file named `natural_disaster_deaths.csv`, which depicts the sum of deaths by natural disaster per year for each country.

In [399]:
#Load all DataFrames for natural disasters
filepath_disasters = './data/natural_disasters/'

#assign country names as primary indexes
df_drought = pd.read_csv(filepath_disasters + 'indicator_drought_killed.csv', index_col = 'Drought killed')
df_earthquake = pd.read_csv(filepath_disasters + 'indicator_earthquake_killed.csv', index_col = 'Earthquake killed')
df_epidemic = pd.read_csv(filepath_disasters + 'indicator_epidemic_killed.csv', index_col = 'Epidemic killed')
df_flood = pd.read_csv(filepath_disasters + 'indicator_flood_killed.csv', index_col = 'Flood killed')
df_storm = pd.read_csv(filepath_disasters + 'indicator_storm_killed.csv', index_col = 'Storm killed')
df_tsunami = pd.read_csv(filepath_disasters + 'indicator_tsunami_killed.csv', index_col = 'Tsunami killed')

In [400]:
#put all DataFrames in a dictionary for iteration
dict_dfs = {'drought killed' : df_drought,
            'earthquake killed' : df_earthquake,
            'epidemic killed' : df_epidemic,
            'flood killed' : df_flood,
            'storm killed' : df_storm,
            'tsunami killed' : df_tsunami
            }

#### Total Forest Area
> I am focusing on total natural forest land per country, and so I will exclude factors that indicate if said forest land is reserved for agricultural production.

> From GapMinder, forest area is described as ‘land under natural or planted stands of trees of at least 5 meters in situ, whether productive or not, and excludes tree stands in agricultural production systems (for example, in fruit plantations and agroforestry systems) and trees in urban parks and gardens.' The dataset `forest_area_sq_km.csv` keeps track of the total forest area from 1990 to 2015.

In [390]:
df_forest = pd.read_csv('./data/forest_area_sq_km.csv', index_col='country')
df_forest.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,...,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0,13500.0
Albania,7890.0,7870.0,7850.0,7830.0,7810.0,7790.0,7770.0,7750.0,7730.0,7710.0,...,7810.0,7800.0,7790.0,7780.0,7760.0,7750.0,7740.0,7730.0,7720.0,7720.0
Algeria,16700.0,16600.0,16500.0,16400.0,16300.0,16200.0,16100.0,16100.0,16000.0,15900.0,...,16100.0,16900.0,17700.0,18400.0,19200.0,19300.0,19300.0,19400.0,19500.0,19600.0
Andorra,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,...,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0
Angola,610000.0,609000.0,607000.0,606000.0,605000.0,604000.0,602000.0,601000.0,600000.0,599000.0,...,590000.0,589000.0,587000.0,586000.0,585000.0,584000.0,582000.0,581000.0,580000.0,579000.0


#### Obesity Rates
> GapMinder provided the age standardized mean for BMI, dividing it into BMI values for men and women. 



In [241]:
df_bmi_male = pd.read_csv('./data/bmi_rates/bmi_male.csv', index_col='Country')
df_bmi_female = pd.read_csv('./data/bmi_rates/bmi_female.csv', index_col='Country')

In [242]:
df_bmi_male.head()

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,21.48678,21.46552,21.45145,21.43822,21.42734,21.41222,21.40132,21.37679,21.34018,21.29845,...,20.75469,20.69521,20.62643,20.59848,20.58706,20.57759,20.58084,20.58749,20.60246,20.62058
Albania,25.22533,25.23981,25.25636,25.27176,25.27901,25.28669,25.29451,25.30217,25.30450,25.31944,...,25.46555,25.55835,25.66701,25.77167,25.87274,25.98136,26.08939,26.20867,26.32753,26.44657
Algeria,22.25703,22.34745,22.43647,22.52105,22.60633,22.69501,22.76979,22.84096,22.90644,22.97931,...,23.69486,23.77659,23.86256,23.95294,24.05243,24.15957,24.27001,24.38270,24.48846,24.59620
Andorra,25.66652,25.70868,25.74681,25.78250,25.81874,25.85236,25.89089,25.93414,25.98477,26.04450,...,26.75078,26.83179,26.92373,27.02525,27.12481,27.23107,27.32827,27.43588,27.53363,27.63048
Angola,20.94876,20.94371,20.93754,20.93187,20.93569,20.94857,20.96030,20.98025,21.01375,21.05269,...,21.31954,21.37480,21.43664,21.51765,21.59924,21.69218,21.80564,21.93881,22.08962,22.25083


#### Murder Rates
> Encompasses number of murders per 100,000 people, accounting for all ages.

In [243]:
df_murder = pd.read_csv('./data/homicide_rates.csv', index_col='Murder per 100,000, age adjusted')

In [244]:
df_murder.head()

,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005
"Murder per 100,000, age adjusted",,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.650731,NaN,3.837318,3.837318
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.972975,8.115989,6.681117,7.326701
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.100290,14.410269,9.806334,4.389205
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.767467,NaN,0.726513,0.726513
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,51.057260,NaN,48.206188,48.206188


## B) Data Cleaning

### Natural Disasters
> To account for all natural disasters that occurred in each country, my goal is to generate a new CSV file where each cell contains the sum of all the natural disaster DataFrames.

Let's examine the columns of each natural disaster DataFrame. There are 195 total countries in the world, so I am expecting there to be at most 195 rows.

In [401]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print(df.index.name, df.shape)

Drought killed (128, 39)
Earthquake killed (97, 39)
Epidemic killed (143, 38)
Flood killed (182, 39)
Storm killed (181, 39)
Tsunami killed (18, 15)


> It can be seen that the DataFrame for tsunamis is the least reported out of all the other natural disasters (accounting for only 18 countries), while the most documented natural disaster is floods, at 181 countries. This adds more ambiguity as to how we should generate our final CSV file accounting for all natural disasters in all countries.

> Now there are two possibilities for approaching this:

Approach 1) The missing countries means that no natural disasters occurred in them, and so it was not necessary to include them in their respective DataFrames.

> Therefore **it is safe to set the values of the missing countries to 0 and generate the final CSV as a sum of all the DataFrames.**

Approach 2) We do not know if any natural disasters occurred in the missing countries.

> Therefore **we should focus only the countries in common who have no missing values.**

I ended up picking **Approach 2** because as can be seen in the above df_drought DataFrame, there are values for 0 in countries and years where no earthquakes happened. The best conclusion I can come for the missing indexed countries is that there has been no data recorded for them, and therefore I cannot assume whether or not earthquakes ever occurred in the missing countries. This logic extends to the rest of the DataFrames for the natural disasters.

#### *DROP NULLS*

In [402]:
#Display number of NaN values in each dataset
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {} : {}'.format(key, df.isna().sum().sum()))

# NaN in drought killed : 0
# NaN in earthquake killed : 0
# NaN in epidemic killed : 0
# NaN in flood killed : 0
# NaN in storm killed : 0
# NaN in tsunami killed : 248


**Only the tsunami dataset has null values.**

In [403]:
#Drop nulls from tsunami dataset
df_tsunami.dropna(inplace=True)

In [404]:
#Confirm changes
df_tsunami.isna().any().any()

False

**But hold up!**
> Before we move on, let's check the first few rows of the tsunami data set:

In [405]:
#Check how many values exist in the tsunami data set
print(df_tsunami.count().sum())

df_tsunami

0


,1979,1980,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007
Tsunami killed,,,,,,,,,,,,,,,


The above results show that only the last DataFrame for tsunamis contains null values. 

> The tsunami dataset is completely empty, which means we'll have to discard it. This makes me come to the conclusion that ***every single row in the data set contained a null value, which means that after dropping all the null values, the dataset became completely empty.***

> I know it's painful to have to discard an entire DataFrame, but at least that is better than incorporating largely unreliable data into our final analysis.

Before moving on, let's update our NumPy array to exclude the tsunami dataset.

In [406]:
#Drop tsunami dataset
dict_dfs.pop('tsunami killed', None)

#check changes
len(dict_dfs)

5

There are only 5 DataFrames in the dictionary instead of 6, so we can move on.

### *Dedupe Data*

In [410]:
#Check for number of duplicate data per row
for key, df in dict_dfs.items():
    print(df.duplicated().sum())

101
21
21
30
29


> But wait, we don't want to drop duplicate data just yet. As we know, many of the DataFrames are populated with rows containing only 0's. Before we drop duplicate rows, let's make sure that they are all actually **zeroes** and not just rows populated with the same repeating values.

In [409]:
#Display all duplicated rows in each DataFrame
for key, df in dict_dfs.items():
    #return True if the dataset contains a value that's not 0
    print(df[df.duplicated(keep=False)].any().any())
    
    
# print(df[1][df[1].duplicated(keep=False)])

False
True
True
False
True


> Some of the DataFrames have values that are greater than 0. But I've come to realize I don't necessarily care about the duplicate values themselves. The real question is: **are they repeating countries?** 


In [411]:
#Check if any of the DataFrames have duplicate index 'country'
for df in np.ndenumerate(dfs_nd_dropped):
    #return True if the dataset contains a repeating country
    print(df[1].index.duplicated().sum().any())

False
False
False
False
False


> None of the DataFrames have repeating countries for indexes. This is good news! Our data has turned out to be much more reliable than expected.

Now we can decide to drop the duplicated rows or not, but I have **ultimately decided not to.** This is an exceptional case. Dropping the rows could harm our data reliability in the end, because dropping duplicates means we would be getting rid of natural disaster scores for entire countries in the long run.

### Move Index to 'Country' Column

In [412]:
for key, df in dict_dfs.items():
    #rename moved column name to 'country'
    new_columns = np.insert(df.columns.values, 0, 'country', axis = 0)
    df.reset_index(inplace = True)
    df.columns = new_columns
    
    #verify that all dfs have column named 'country'
    print(np.any(df.columns.values == 'country'))

True
True
True
True
True


#### *FIX DATA TYPES*

Counting the number of natural disaster occurrences means handling **discrete variables**, so it would make the most sense to convert all of the values in every DataFrame to integers.

In [413]:
#Display data types of each DataFrame
for df in np.ndenumerate(dfs_nd_dropped):
    print(np.unique(df[1].dtypes))

[dtype('int64')]
[dtype('int64')]
[dtype('int64')]
[dtype('int64')]
[dtype('int64')]


All of the columns contain only integer values, so we do not have to perform any data type conversion or extraction

Before generating a final DataFrame containing the sum of each natural disaster occurrence per country and year, **let's first check to see that their year ranges are consistent.**

In [415]:
from functools import reduce

column_names = []
for i in range(len(dfs_nd)):
    column_names.append(dfs_nd[i].columns.values)

#find years in common
reduce(np.intersect1d, column_names)

array(['1970', '1971', '1972', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008'], dtype=object)

The year 1973 is missing from one or more of the DataFrames. After manually scrolling through the previous years, we can see that `df_epidemic` is the one with the missing year.

In [417]:
#verify 1973 does not exist
df_epidemic.columns[1:5]

Index(['1970', '1971', '1972', '1974'], dtype='object')

***MELT DATA***

In [432]:
for key, df in dict_dfs.items():
    pd.melt(df_drought, id_vars = 'country', var_name = 'year', value_name = 'drought_killed')



#### *OUTER MERGE DATASETS*
> **Main idea:** We are using an ***outer merge*** because our data implies that each country in each DataFrame satisfies the following conditions:

1) contains reliable non-null data

2) represents the true number of its respective natural disaster per year

> Consider that we are merging df1 and df2.

If df1 contains countries that df2 does not AND df2 contains countries that df1 does not, we want all of those countries to show up in the final result.

If df2 contains years that df2 does not AND df2 contains years that df1 does not, we want all of those years to show up in the final result anyway.

#### *CLASSIFY FREQUENCY AS 'LOW', 'MEDIUM', OR 'HIGH'*

### Obesity Rates
> Though generally it may seem that a higher BMI indicates a 'healthier' weight for male, [this article](https://signup.weightwatchers.co.uk/util/art/index_art.aspx?art_id=31901&tabnum=1&sc=803&subnav=Science+Library%3A+Health+and+Weight) clarifies that BMI rates above 25 would still indicate ill health: an equal expectation for both men and women. Based on this fast, I decided it was safe to generate a new CSV, 'bmi_indicator,' for both men and women, by filling each slot with the mean BMI value for men and women.

In [38]:
#Creating 'natural_disasters_killed.csv'

<a id='eda'></a>
# III) Exploring the Data

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### How much have global obesity rates changed in countries with high forest area vs. countries with low forest area?

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Does a country’s likelihood of experiencing a natural disaster affect the homicidal tendencies of its citizens?

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
# IV) Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])